# Wykład numer 1

Technika Few-Shots

Główna idea:
- Wybrać kilka demnstracji
- Wkleić je do prompta

Prawdopodobieństwo

"markdown.extension.preview.autoShowPreviewToSide": true


P(w<sub>1</sub> ... w<sub>n</sub>) = P(w<sub>1</sub>)P(w<sub>2</sub> | w<sub>1</sub>) <sub></sub>

{
    "siema": "Czesc",
    "surrname": "Co tam",

}


<sub>123</sub>
term
: siemka

- [x] Siemka
- []

:joy:

<sup>2</sup>
H~2~0



<div align="center" style="color: white">

## Wzór na Soft Max

</div>
<div style="color: white">


<p align="center" style="color: white"> Wektor Logits'ów</p>

$$
    Z = \begin{pmatrix} z_1 & z_2 & ... & z_{n-1} & z_n \end{pmatrix} 
$$ 

$$
    softmax({z_k}) = \frac{e^{z_k}}{\sum_{i=1}^n e^{z_i}}
$$

$$ 
    p_k = \frac{e^{z_k}}{\sum_{i=1}^n e^{z_i}}
$$

</div>

<div align="center" style="color: white">

## Wzór na Log Soft Max

</div>
<div style="color: white">

$$
    logsoftmax({z_k}) = log(\frac{e^{z_k}}{\sum_{i=1}^n e^{z_i}})
$$

$$ 
    logsoftmax({z_k}) = log(e^{z_k}) - log(\sum_{i=1}^n e^{z_i})
$$

$$
    logsoftmax({z_k}) = z_k - log(\sum_{i=1}^n e^{z_i}) 
$$


</div>

<div align="center">
    <h2>
        Kod z wykładu
    </h2>
</div>

In [43]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
import random
import numpy as np

model_name = 'flax-community/papuGaPT2'
device = 'cuda'
device = 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    print(logp)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    print(logp_label)
    return logp_label
    
            
def sentence_prob(sentence_txt):
    input_ids = tokenizer(sentence_txt, return_tensors='pt')['input_ids'].to(device)
    with torch.no_grad():
        output = model(input_ids=input_ids)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
        seq_log_probs = torch.sum(log_probs)
    return seq_log_probs.cpu().numpy()  
    
sentences = [
  'Ala ma kota.',  
]

print ()
for s in sentences:
    print (s, sentence_prob(s))
    


tensor([[[-22.9720, -25.3151, -25.6619,  ..., -14.4614, -18.1689, -13.7099],
         [-24.8636, -26.9614, -28.9603,  ..., -17.5621, -16.1743, -12.8644],
         [-25.4614, -24.5363, -25.7523,  ..., -21.0725,  -9.4578, -15.4223],
         [-27.2302, -31.5633, -30.7097,  ..., -17.6647, -14.6530, -13.6264]]])
tensor([[-7.5045, -4.2623, -6.6108, -2.0110]])
Ala ma kota. -20.388588


In [39]:
import numpy as np
from math import e, pow

logits = [2.5, 1.0, 4.2, -0.3, 0.7]
print(np.sum([2.5, 1.0, 4.2, -0.3, 0.7]))
a = torch.tensor([[2.5, 1.0, 4.2, -0.3, 0.7]])
print(torch.softmax(a, dim=1))
b = torch.tensor([2.0, 1.0, 0.1])

print('log: ', torch.log_softmax(b, dim=0))

s = np.sum([pow(e, l) for l in logits])
print(s)
for i in range(len(logits)):
    logits[i] = pow(e, logits[i]) / s

print(logits)
print(np.sum(logits))


8.1
tensor([[0.1444, 0.0322, 0.7907, 0.0088, 0.0239]])
log:  tensor([-0.4170, -1.4170, -2.3170])
84.34167775823985
[np.float64(0.14444215818926234), np.float64(0.03222940188895495), np.float64(0.7906687750755603), np.float64(0.008783536685210685), np.float64(0.0238761281610116)]
0.9999999999999999


<div align="center"> 

# Beam Search

</p>

<div align="center">

# Perplexity (miara nieokreśloności)

</div>

---
Intuicje
1. To co się zdaza, powinno mieć wysokie prawdopodobienstwo
2. Gdy dobrze perzewidumey koleje slowo ( na podstawie pelnego prefiksu) to jestesmy w stanie dobrze kompresowwac tekst
3. Oczywiście powinniśmy dzielic korpus na czesc przeszla zdarzyla sie i przyszla zdarzy sie chemy jej dac spore prawdopodobienstwo, ale jej nie znamy
---
Wzór

$$
    PP(w_1 ... w_N) = P(w_1 ... w_N)^{-\frac{1}{N}}
$$

---

Mozna rozumieć perplexity jako średni wazony współczynnik rozgałęzienia języka